UPLOADING THE DATASET FROM KAGGLE

In [ ]:
# upload kaggle API token into the folder which comes up on connecting
! pip install kaggle 
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download vivekrathi055/sentiment-analysis-on-financial-tweets

  0% 0.00/2.42M [00:00<?, ?B/s]
100% 2.42M/2.42M [00:00<00:00, 79.2MB/s]


In [ ]:
! unzip sentiment-analysis-on-financial-tweets

Archive:  sentiment-analysis-on-financial-tweets.zip
  inflating: stockerbot-export1.csv  
  inflating: tweet_sentiment.csv     


In [ ]:
# Mount the Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/kaggle.json"
! cd /content

In [ ]:
import pandas as pd
dataset = pd.read_csv('/content/stockerbot-export1.csv')
dataset.head()

,id,text,timestamp,source,symbols,company_names,url,verified
0,1.0197E+18,VIDEO: “I was in my office. I was minding my o...,Wed Jul 18 21:33:26 +0000 2018,GoldmanSachs,GS,The Goldman Sachs,https://twitter.com/i/web/status/1019696670777...,TRUE
1,1.01971E+18,The price of lumber $LB_F is down 22% since hi...,Wed Jul 18 22:22:47 +0000 2018,StockTwits,M,Macy's,https://twitter.com/i/web/status/1019709091038...,TRUE
2,1.01971E+18,Who says the American Dream is dead? https://t...,Wed Jul 18 22:32:01 +0000 2018,TheStreet,AIG,American,https://buff.ly/2L3kmc4,TRUE
3,1.01972E+18,Barry Silbert is extremely optimistic on bitco...,Wed Jul 18 22:52:52 +0000 2018,MarketWatch,BTC,Bitcoin,https://twitter.com/i/web/status/1019716662587...,TRUE
4,1.01972E+18,How satellites avoid attacks and space junk wh...,Wed Jul 18 23:00:01 +0000 2018,Forbes,ORCL,Oracle,http://on.forbes.com/6013DqDDU,TRUE


PRE PROCESSING THE DATASET USING INBUILT LIBRARY 

In [ ]:
tweets = dataset["text"].tolist() # convert tweets to a list 

In [ ]:
! pip install tweet-preprocessor
clean = []
import preprocessor as p

def preproc(tweet):
  clean_tweet = p.clean(tweet)
  return clean_tweet

# Cleaning
for i in tweets:
  clean.append(preproc(i))

In [40]:
print(clean[0])

VIDEO: I was in my office. I was minding my own business... David Solomon tells $GS interns how he learned he wa


PRE PROCESSING USING REGEX ETC.

In [41]:
import regex as re
no_link = []

def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

for i in tweets:
  no_link.append(remove_links(i))

print(no_link[0])

VIDEO: “I was in my office. I was minding my own business...” –David Solomon tells $GS interns how he learned he wa… 


In [ ]:
# CONVERT TO LOWER CASE
import re
low = []

def lower(tweets):
  data = re.sub(' +',' ',tweets)
  lower_text = data.lower()
  return lower_text

for i in no_link:
    low.append(lower(i))

In [42]:
print(low[0])

video: “i was in my office. i was minding my own business...” –david solomon tells $gs interns how he learned he wa… 


In [43]:
# REMOVE PUNCTUATION
def remove_punctuation(words):
  new_words = []
  for word in words:
    new_word = re.sub(r'[^\w\s]', '', (word))
    if new_word != '':
       new_words.append(new_word)
  return new_words

new = remove_punctuation(low)
print(new[0])

video i was in my office i was minding my own business david solomon tells gs interns how he learned he wa 


In [44]:
# REMOVE USERS 
no_user = []

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    return tweet

for i in new:
    no_user.append(remove_users(i))

print(no_user[0])

video i was in my office i was minding my own business david solomon tells gs interns how he learned he wa 


In [45]:
# REMOVE HASHTAGS
no_hash = []

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    return tweet

for i in no_user:
    no_hash.append(remove_hashtags(i))

print(no_hash[0])

video i was in my office i was minding my own business david solomon tells gs interns how he learned he wa 


In [46]:
# REMOVE SPACES AND DIGITS 
data_final = []

def spaces_digits(tweet):
    tweet = re.sub('\s+', ' ', tweet)  # remove multi spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    return tweet

for i in no_hash:
  data_final.append(spaces_digits(i))

print(data_final[0])

video i was in my office i was minding my own business david solomon tells gs interns how he learned he wa 


In [ ]:
# REMOVE STOP WORDS

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

res = [] # resultant list of cleaned strings

for s in data_final:
  lst_string = [s][0].split()
  
  no_stpwords_string="" #new string created 
  for i in lst_string:
    if not i in stop_words:
        no_stpwords_string += i + ' '
  no_stpwords_string = no_stpwords_string[:-1]
  res.append(no_stpwords_string)

print(res[0]) # example cleaned string 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
video office minding business david solomon tells gs interns learned wa


WORD TOKENISATION

In [ ]:
from nltk.tokenize import regexp_tokenize
  
res_fin = [sub.split() for sub in res] #creates a list of lists for all tweets tokenised
print(res_fin[0])

['video', 'office', 'minding', 'business', 'david', 'solomon', 'tells', 'gs', 'interns', 'learned', 'wa']


WORD STEMMING

In [ ]:
from nltk.stem import PorterStemmer
porter = PorterStemmer()
stemmed = []
for words in res_fin:
    temp = [porter.stem(i) for i in words]
    stemmed.append(temp)
print(stemmed[0])    

['video', 'offic', 'mind', 'busi', 'david', 'solomon', 'tell', 'gs', 'intern', 'learn', 'wa']


WORD LEMMATISATION

In [ ]:
import nltk
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
# import these modules
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

lemmatised = []
for words in stemmed:
  temp = [lemmatizer.lemmatize(i) for i in words]
  lemmatised.append(temp)
print(lemmatised[0])  

['video', 'offic', 'mind', 'busi', 'david', 'solomon', 'tell', 'g', 'intern', 'learn', 'wa']


INVERTED INDEX CREATION 

In [34]:
from collections import defaultdict
def create_index (data):
  index = defaultdict(list)
  for i, tokens in enumerate(data):
    for token in tokens:
      index[token].append(i)
  return index

In [60]:
index = create_index(lemmatised)
print("hello : ",index['hello'])

print("\n")
print("EXAMPLE KEYS: ")
# show a few example keys - first 10 
for i in list(index)[0:10]:
  print(i,',',index[i])

hello :  [15988, 16343, 16487, 16876, 16894, 17091, 18753, 18956, 19017, 19105, 19617, 19874, 20855, 20966, 24252, 24655, 28258, 28420]


EXAMPLE KEYS: 
video , [0, 800, 1048, 1206, 1327, 1447, 1588, 1597, 1776, 1870, 1924, 2118, 2355, 2439, 2552, 2703, 2704, 3418, 3419, 3597, 3604, 3624, 3626, 3683, 4541, 4550, 5186, 5490, 5508, 5618, 5670, 5778, 5864, 6148, 6672, 6747, 6804, 8141, 8208, 8373, 8385, 8386, 8387, 8388, 8389, 8390, 8391, 8392, 8394, 8395, 8396, 8397, 8398, 8399, 8410, 8485, 8488, 8850, 8892, 10499, 10653, 10663, 10678, 11804, 11824, 12018, 12049, 12220, 12302, 13653, 14539, 14935, 15077, 15326, 15690, 16196, 16237, 16527, 16597, 17437, 17993, 18143, 18684, 18907, 19089, 19348, 19532, 20099, 20207, 20322, 20943, 21006, 22183, 22603, 22611, 22622, 22698, 22700, 22755, 22768, 23407, 23424, 23468, 23558, 24073, 24148, 24347, 24693, 24721, 24886, 24934, 25450, 25761, 25853, 25858, 26089, 26125, 26257, 26334, 26347, 26377, 26483, 26593, 26674, 26683, 27111, 27135, 27702, 27864